In [0]:
dbutils.fs.unmount('/mnt/retail')

/mnt/retail has been unmounted.
Out[41]: True

In [0]:
#Connect to ADL gen 2 using Access Key
dbutils.fs.mount(
  source = "wasbs://retail-4ysales@projectpro.blob.core.windows.net",
  mount_point = "/mnt/retail",
  extra_configs = {"fs.azure.account.key.projectpro.blob.core.windows.net" : "0iLQdIZE7A5FddGLF4B4z4BglCiOQt4IGrrPVZ2XSpQGOS3ouAJ7zOUDqeTBKpAHyL/6SI2epzVH+AStPV1v+w=="})

Out[43]: True

In [0]:
#folders in retail 4 years container
for file in dbutils.fs.ls('/mnt/retail'):
    print(file.name)

Processed Data/
RwaData/


In [0]:
#folders in retail 4 years container

def list_all_subfolders(path, depth=0):
    try:
        # List all items at the current path
        items = dbutils.fs.ls(path)
        
        # Iterate through the items
        for item in items:
            
            # Check if the item is a directory
            if item.isDir():
                # Print the directory path
                print("  " * depth + item.path)
                
                # Recursively list the contents of the directory
                list_all_subfolders(item.path, depth + 1)
            else:
                print("  " * depth + item.name)

                
    except Exception as e:
        print(f"Error accessing path {path}: {str(e)}")

# Example usage:
path_to_list = "/mnt/retail"  # Example path, adjust according to your needs
list_all_subfolders(path_to_list)


dbfs:/mnt/retail/Processed Data/
dbfs:/mnt/retail/RwaData/
  dbfs:/mnt/retail/RwaData/Logs/
  customer
  e6dd8201-c874-4ef5-9d6c-88c2970754fd
  products
  stores


In [0]:
mnt_rawdata = '/mnt/retail/RwaData/'

for data in dbutils.fs.ls(mnt_rawdata):
    if data.name =='customer':
        df_customer = spark.read.option("Header",True).csv(data.path,inferSchema=True)
    if data.name =='products':
        df_products = spark.read.option("Header",True).csv(data.path,inferSchema=True)
    if data.name == 'stores':
        df_stores = spark.read.option("header",True).csv(data.path, inferSchema=True)


In [0]:
df_customer.show(3)

+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
|Customer_ID|  Customer_Name|  Segment|      Country|           City|     State|Postal_Code|Region|Age|
+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
|   CG/12520|    Claire Gute| Consumer|United States|      Henderson|  Kentucky|      42420| South| 42|
|   DV/13045|Darrin Van Huff|Corporate|United States|    Los Angeles|California|      90036|  West| 47|
|   SO/20335| Sean O'Donnell| Consumer|United States|Fort Lauderdale|   Florida|      33311| South| 19|
+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---+
only showing top 3 rows



In [0]:
#Data frames row count
print("Customer_Count: ",df_customer.count(),
       "\n Product_Count: ",df_products.count(),
        "\n Stores_Count: ", df_stores.count())

Customer_Count:  793 
 Product_Count:  1861 
 Stores_Count:  9800


In [0]:
# Assuming df is your PySpark DataFrame
df_customer.createOrReplaceGlobalTempView("df_customer_GloabalTempBiew")

# Assuming df is your PySpark DataFrame
df_products.createOrReplaceGlobalTempView("df_products_GloabalTempBiew")

#df_stores.write.format("parquet").save("/tmp/df_stores_wr")

In [0]:
df_customer.select('Segment').distinct().display()

Segment
Consumer
Home Office
Corporate


In [0]:

df_stores.describe().show()
df_products.describe().show()

+-------+------------------+--------------+--------------+-----------+---------------+------------------+--------------------+
|summary|            Row_ID|      Order_ID|     Ship_Mode|Customer_ID|     Product_ID|             Sales|            Discount|
+-------+------------------+--------------+--------------+-----------+---------------+------------------+--------------------+
|  count|              9800|          9800|          9800|       9800|           9800|              9770|                9800|
|   mean|            4900.5|          null|          null|       null|           null|3472113.5476458548|0.020096938775510883|
| stddev|2829.1606529145706|          null|          null|       null|           null| 9412258.529173262|0.008120733444776835|
|    min|                 1|CA-2015-100006|   First Class|   AA/10315|FUR-BO-10000112|           21120.0|                0.01|
|    max|              9800|US-2018-169551|Standard Class|   ZD/21925|TEC-PH-10004977|        3.395772E8|      

In [0]:
df_customer.describe().show()

+-------+-----------+------------------+-----------+-------------+----------+---------+------------------+-------+------------------+
|summary|Customer_ID|     Customer_Name|    Segment|      Country|      City|    State|       Postal_Code| Region|               Age|
+-------+-----------+------------------+-----------+-------------+----------+---------+------------------+-------+------------------+
|  count|        793|               793|        793|          793|       793|      793|               793|    793|               793|
|   mean|       null|              null|       null|         null|      null|     null|55229.717528373265|   null|33.746532156368225|
| stddev|       null|              null|       null|         null|      null|     null| 31679.22381580119|   null| 8.628123278990753|
|    min|   AA/10315|     Aaron Bergman|   Consumer|United States|     Akron|  Alabama|              1841|Central|                19|
|    max|   ZD/21925|Zuschuss Donatelli|Home Office|United Sta

In [0]:
source_df = [df_customer,df_products, df_stores]
print(type(source_df[0]))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
def lust_all_distinct_values(src):
    for data_frame in src:
        print(data_frame, " Columns")
        dataframe_columns = data_frame.columns
        for col in dataframe_columns:
            print(col, " Distinct Values : ")
            data_frame.select(col).distinct().show(5)
lust_all_distinct_values(source_df)

DataFrame[Customer_ID: string, Customer_Name: string, Segment: string, Country: string, City: string, State: string, Postal_Code: int, Region: string, Age: int]  Columns
Customer_ID  Distinct Values : 
+-----------+
|Customer_ID|
+-----------+
|   IM/15070|
|   SO/20335|
|   AA/10480|
|   CG/12520|
|   DV/13045|
+-----------+
only showing top 5 rows

Customer_Name  Distinct Values : 
+---------------+
|  Customer_Name|
+---------------+
|    Claire Gute|
| Sean O'Donnell|
|   Irene Maddox|
|Darrin Van Huff|
|Brosina Hoffman|
+---------------+
only showing top 5 rows

Segment  Distinct Values : 
+-----------+
|    Segment|
+-----------+
|   Consumer|
|Home Office|
|  Corporate|
+-----------+

Country  Distinct Values : 
+-------------+
|      Country|
+-------------+
|United States|
+-------------+

City  Distinct Values : 
+---------------+
|           City|
+---------------+
|    Los Angeles|
|Fort Lauderdale|
|        Seattle|
|      Henderson|
|        Concord|
+---------------+
onl